## importando requisitos

In [12]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

### lendo e verificando dados

In [13]:
df = pd.read_csv('datasets/users_behavior.csv')
df.info()
print()
print(df.head())
print()
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64


### separando treinamento validação e teste e definindo features e target

In [14]:
df_train, df_temp= train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test= train_test_split(df_temp,test_size=0.5,random_state =12345)

In [15]:
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(1928, 5)
(643, 5)
(643, 5)


In [16]:
def set_fet_targ(df):
    features = df.drop(['is_ultra'],axis=1)
    target = df['is_ultra']
    return features,target

fet_train,targ_train = set_fet_targ(df_train)
fet_valid,targ_valid = set_fet_targ(df_valid)
fet_test,targ_test   = set_fet_targ(df_test)



## Testando modelos

### Decision Tree

In [17]:
for i in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model.fit(fet_train,targ_train)
    print('profundidade: ',i)
    print('acuracia train',model.score(fet_train,targ_train))
    print('acuracia valid',model.score(fet_valid,targ_valid))
    print()

print('acuracia modelo test',model.score(fet_test,targ_test))


profundidade:  1
acuracia train 0.7577800829875518
acuracia valid 0.7542768273716952

profundidade:  2
acuracia train 0.7878630705394191
acuracia valid 0.7822706065318819

profundidade:  3
acuracia train 0.8075726141078838
acuracia valid 0.7853810264385692

profundidade:  4
acuracia train 0.8106846473029046
acuracia valid 0.7791601866251944

profundidade:  5
acuracia train 0.8200207468879668
acuracia valid 0.7791601866251944

acuracia modelo test 0.7838258164852255


No modelo de arvore de decisão, podemos perceber que a partir da profundidade 2 ja possui uma acuracia de 0.77 no conjunto de validação, acima do minimo esperado (0.75), ao realizar o teste de acuracia no modelo de teste obtivemos uma acuracia de 0.78, maior do que o minimo esperado.

In [18]:
model = DecisionTreeClassifier(random_state=12345, min_samples_leaf=3)
model.fit(fet_train,targ_train)
print('acuracia modelo train, alterando samples',model.score(fet_train,targ_train))
print('acuracia modelo test, alterando samples',model.score(fet_test,targ_test))

acuracia modelo train, alterando samples 0.9258298755186722
acuracia modelo test, alterando samples 0.7527216174183515


In [19]:
model = DecisionTreeClassifier(random_state=12345, min_samples_split=4)
model.fit(fet_train,targ_train)
print('acuracia modelo train, alterando split',model.score(fet_train,targ_train))
print('acuracia modelo test, alterando split',model.score(fet_test,targ_test))

acuracia modelo train, alterando split 0.9761410788381742
acuracia modelo test, alterando split 0.7325038880248833


In [20]:
model = DecisionTreeClassifier(random_state=12345, min_samples_leaf=2, min_samples_split=4)
model.fit(fet_train,targ_train)
print('acuracia modelo train, alterando samples e split',model.score(fet_train,targ_train))
print('acuracia modelo test, alterando samples e split',model.score(fet_test,targ_test))

acuracia modelo train, alterando samples e split 0.9491701244813278
acuracia modelo test, alterando samples e split 0.7589424572317263


A alteração dos demais paramentros se mostrou menos eficiente com uma queda na acuracia, onde alterando apenas o split obtivemos um resultado menor do que o minimo esperado

### Random Forest

In [21]:
model = RandomForestClassifier(random_state=12345)

param_grid = {
    'n_estimators': [1, 2, 3, 4, 5],
    'max_depth': [None,1,2,3],
    'min_samples_split': [2, 3, 4]
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(fet_train, targ_train)
print("Melhores hiperparâmetros:", grid_search.best_params_)

grid_search.score(fet_test, targ_test)

Melhores hiperparâmetros: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 4}


0.7729393468118196

Podemos perceber o mesmo padrão do modelo de decisão porem, com uma acuracia no modelo de teste da floresta menor do que o da arvore de decisão

### Logistic Regression

In [22]:
scaler = StandardScaler()

fet_train = scaler.fit_transform(fet_train)
fet_valid = scaler.fit_transform(fet_valid)
fet_test = scaler.fit_transform(fet_test)

model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(fet_train,targ_train)
print('acuracia train',model.score(fet_train,targ_train))
print('acuracia valid',model.score(fet_valid,targ_valid))

print()
print('acuracia test',model.score(fet_test,targ_test))

acuracia train 0.7531120331950207
acuracia valid 0.7573872472783826

acuracia test 0.7418351477449455


No modelo de Regressão Logistica podemos notar que a acuracia do conjunto de teste foi menor do que o minimo especificado

## Conclusão

Os modelos que apresentaram melhor resultado foram o de arvore de decisão e o de floresta respectivamente 0.78 e 0.77, o modelo de regressão logistica foi o menos eficiente com 0.74